In [37]:
import requests
import os
import pandas as pd
import shutil

In [3]:
pokemon_url="https://pokeapi.co/api/v2/pokemon"
pokemon_start=1
pokemon_end=1025

In [65]:
def get_pokemon_img_links(pokemon_url,pokemond_id):
    response = requests.get(f"{pokemon_url}/{pokemond_id}")
    if response.status_code == 200:
        data = response.json()
        sprites = data.get('sprites', {})
        types_obj=data.get('types',[])
        
        types=[ type["type"]["name"] for type in types_obj ]
        name=data.get("name")
        image_links = {
            "back_default": sprites.get('back_default'),
            "back_female": sprites.get('back_female'),
            "back_shiny": sprites.get('back_shiny'),
            "back_shiny_female": sprites.get('back_shiny_female'),
            "front_default": sprites.get('front_default'),
            "front_female": sprites.get('front_female'),
            "front_shiny": sprites.get('front_shiny'),
            "front_shiny_female": sprites.get('front_shiny_female')
        }
        pokemon_details={
            "id":pokemond_id,
            "name":name,
            "types":types,
            "image_links":image_links
        }
        return pokemon_details
    else:
        print("Failed to fetch data from API")
        return None

In [68]:
new_dataset_dir="../dataset-v2"
csv_path="../csv"

In [101]:
def create_img_csv(pokemon_details, dataset_dir):
    '''
    ->create main folder->dataset-v2
    ->sub folders with name of the pokemon
    ->write img_path of pokemon img in a csv with its respective types and id 
    {
    "id":
    "name":
    "type1":
    "type2":
    "img_path":
    }
    '''
    pokemon_name = pokemon_details["name"]  
    pokemon_id = pokemon_details["id"]
    img_links = pokemon_details["image_links"]
    types = pokemon_details["types"]
    print(pokemon_name, pokemon_id)
    if not os.path.isdir(f"{dataset_dir}/{pokemon_name}"):
        os.makedirs(f"{dataset_dir}/{pokemon_name}")

    img_suffix_num = 0  # name ->bubasaur_0.png bulbasaur_1.png

    pokemon_csv_data = []
    pokemon = {}
    for img_link in img_links:
        img_path = f"{dataset_dir}/{pokemon_name}/{pokemon_name}_{img_suffix_num}.png"
        if img_links[img_link] is None:
            continue
        response = requests.get(img_links[img_link], stream=True)
        with open(img_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response
        pokemon = {
            "name": pokemon_name,
            "id": pokemon_id,
            "image_path": img_path,
            "type1":types[0]
        }
        if len(types) == 2:
            pokemon["type2"] = types[1]

        
        pokemon_csv_data.append(pokemon)
        img_suffix_num +=1
    return pokemon_csv_data


In [102]:
gyarados=get_pokemon_img_links(pokemon_url,131)
print(gyarados)
create_img_csv(gyarados,new_dataset_dir)

{'id': 131, 'name': 'lapras', 'types': ['water', 'ice'], 'image_links': {'back_default': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/back/131.png', 'back_female': None, 'back_shiny': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/back/shiny/131.png', 'back_shiny_female': None, 'front_default': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/131.png', 'front_female': None, 'front_shiny': 'https://raw.githubusercontent.com/PokeAPI/sprites/master/sprites/pokemon/shiny/131.png', 'front_shiny_female': None}}
lapras 131


[{'name': 'lapras',
  'id': 131,
  'image_path': '../dataset-v2/lapras/lapras_0.png',
  'type1': 'water',
  'type2': 'ice'},
 {'name': 'lapras',
  'id': 131,
  'image_path': '../dataset-v2/lapras/lapras_1.png',
  'type1': 'water',
  'type2': 'ice'},
 {'name': 'lapras',
  'id': 131,
  'image_path': '../dataset-v2/lapras/lapras_2.png',
  'type1': 'water',
  'type2': 'ice'},
 {'name': 'lapras',
  'id': 131,
  'image_path': '../dataset-v2/lapras/lapras_3.png',
  'type1': 'water',
  'type2': 'ice'}]

In [103]:
def create_dataset(pokemon_url,dataset_dir,pokemon_csv_path):
    csv_data=[]
    for pokemon_id in range(pokemon_start,pokemon_end+1):
        pokemon_details=get_pokemon_img_links(pokemon_url,pokemon_id)
        if pokemon_details is not None:
            pokemon_dict=create_img_csv(pokemon_details,dataset_dir)
            for pokemon in pokemon_dict:
                csv_data.append(pokemon)
            
        
    pokemon_df=pd.DataFrame(csv_data)
    return pokemon_df
            

In [104]:
pokemon_df=create_dataset(pokemon_url,new_dataset_dir,csv_path)

bulbasaur 1
ivysaur 2
venusaur 3
charmander 4
charmeleon 5
charizard 6
squirtle 7
wartortle 8
blastoise 9
caterpie 10
metapod 11
butterfree 12
weedle 13
kakuna 14
beedrill 15
pidgey 16
pidgeotto 17
pidgeot 18
rattata 19
raticate 20
spearow 21
fearow 22
ekans 23
arbok 24
pikachu 25
raichu 26
sandshrew 27
sandslash 28
nidoran-f 29
nidorina 30
nidoqueen 31
nidoran-m 32
nidorino 33
nidoking 34
clefairy 35
clefable 36
vulpix 37
ninetales 38
jigglypuff 39
wigglytuff 40
zubat 41
golbat 42
oddish 43
gloom 44
vileplume 45
paras 46
parasect 47
venonat 48
venomoth 49
diglett 50
dugtrio 51
meowth 52
persian 53
psyduck 54
golduck 55
mankey 56
primeape 57
growlithe 58
arcanine 59
poliwag 60
poliwhirl 61
poliwrath 62
abra 63
kadabra 64
alakazam 65
machop 66
machoke 67
machamp 68
bellsprout 69
weepinbell 70
victreebel 71
tentacool 72
tentacruel 73
geodude 74
graveler 75
golem 76
ponyta 77
rapidash 78
slowpoke 79
slowbro 80
magnemite 81
magneton 82
farfetchd 83
doduo 84
dodrio 85
seel 86
dewgong 87
gri

In [109]:
pokemon_df.shape

(4148, 5)

In [111]:
pokemon_df.to_csv(f"pokemon-v2.csv")